In [1]:
import os


In [2]:
%pwd

'd:\\Piyush\\End-to-end-classification-project-pipeline-\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Piyush\\End-to-end-classification-project-pipeline-'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.el
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config


In [16]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [17]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config  


    
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)


    # This function _prepare_full_model is responsible for preparing a full model based on a given base model and some additional parameters
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        '''
        Freezing Layers:
        If freeze_all is True, it loops through all layers of the model and sets trainable property to False, effectively freezing all layers.
        If freeze_all is False and freeze_till is not None and greater than 0, it freezes layers up to the specified index. This is done by iterating through the layers of the model except for the last freeze_till layers and setting them as not trainable.
        '''
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        '''
        Adding Layers:

        tf.keras.layers.Flatten() creates a Flatten layer. This layer is responsible for converting the multi-dimensional output from the preceding layer into a one-dimensional array. It effectively 'flattens' the input.
        model.output represents the output tensor of the previous layer or model. In this case, it's the output tensor of the model passed into the function.
        tf.keras.layers.Flatten()(model.output) applies the Flatten layer to the model.output, transforming it into a one-dimensional array. The result is assigned to flatten_in.
        
        tf.keras.layers.Dense() creates a Dense layer. Dense layers are fully connected layers, meaning each neuron in the layer receives input from every neuron in the previous layer.
        units=classes specifies the number of units (neurons) in this Dense layer. The classes variable likely represents the number of classes in the classification task. Each neuron in this layer will correspond to one class.
        activation="softmax" sets the activation function for the neurons in this Dense layer to softmax. Softmax is commonly used in the output layer of a classification model as it converts the raw output scores into probabilities, making it easier to interpret.
        tf.keras.layers.Dense(units=classes, activation="softmax")(flatten_in) applies the Dense layer to the flatten_in tensor, which is the flattened output from the previous layer. This completes the construction of the output layer of the model, and the result is assigned to prediction.
        In summary, these lines of code create the final layers of the model: a Flatten layer to flatten the output of the previous layer into a one-dimensional array, followed by a Dense layer with softmax activation, which outputs probabilities for each class in a classification task.
        '''
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        '''It creates a full model using the input of the base model and the prediction layer created above.'''
        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )
        '''It compiles the full model with SGD optimizer, categorical crossentropy loss function, and accuracy metric.'''
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        '''It prints a summary of the full model, showing the architecture and the number of parameters.'''
        full_model.summary()
        return full_model
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    


In [18]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-05-06 07:27:12,592: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-06 07:27:12,598: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-06 07:27:12,601: INFO: common: created directory at: artifacts]
[2024-05-06 07:27:12,604: INFO: common: created directory at: artifacts/prepare_base_model]
58889256/58889256 [==============================] - 5s 0us/step
[2024-05-06 07:27:19,687: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                 

Freezing a layer in a neural network model means preventing its weights from being updated during the training process. When you freeze a layer, its weights remain constant and are not adjusted based on the optimization algorithm's updates (e.g., gradient descent) during training.

Freezing layers is a common technique in transfer learning and fine-tuning scenarios, where you may have a pre-trained model and want to adapt it to a new task or dataset. By freezing certain layers, you preserve the knowledge encoded in those layers while allowing the model to learn new patterns in the unfrozen layers.

There are a few reasons why you might want to freeze layers:

Transfer Learning: When you have a pre-trained model trained on a large dataset (e.g., ImageNet), you might want to use it as a starting point for a new task. Freezing the early layers, which capture generic features like edges and textures, allows you to retain these learned features while training only the later layers on your specific task and dataset.
Preventing Overfitting: Freezing layers can help prevent overfitting, especially when you have a limited amount of training data. By freezing some of the layers and reducing the number of trainable parameters, you can reduce the risk of overfitting to the training data.
Computational Efficiency: Training deep neural networks can be computationally expensive, especially if the model has many parameters. Freezing layers reduces the computational cost since the frozen layers don't require gradient computations during training.
When freezing layers, it's essential to consider which layers to freeze and which ones to leave trainable. Typically, you freeze the early layers (closer to the input) and leave the later layers (closer to the output) trainable, as the early layers capture more general features that are likely relevant to your task. However, the specific strategy may vary depending on the dataset, model architecture, and task at hand.





